In [1]:
#FCF yield & Momentum investment strategy
#Nifty100 stocks pool

import yfinance as yf
import pandas as pd
import numpy as np


In [3]:
# Load CSV data into a DataFrame
file_path = 'stock_data.CSV'
result_df = pd.read_csv(file_path)
result_df = result_df.dropna()
# Display the DataFrame
result_df


,S.No.,Name,priceToEarnings,earningsYield,enterpriseValue,volume,profits,debt,roa,bookValue,...,ebidt,ebidtYield,freeCashFlowToTotaldebt,freeCashFlowYield,liquidity,equity,bookToMarket,assets,accrualCF,freeCashFlowYield5Yrs
0,1,ABB.NS,96.44,1.45,115608.37,335597,1248.18,48.98,12.29,280.54,...,1791.47,0.015496,23.85463454,0.010107,0.003719,42.38,0.000352,11001.19,0.007252,0.026181
1,2,ADANIENSOL.NS,102.48,2.97,152820.78,2917461,1165.29,36030.81,2.53,106.66,...,6101.13,0.039923,-0.025678579,-0.006054,0.000399,1115.49,0.009341,55729.96,0.037511,0.018845
2,3,ADANIENT.NS,100.55,2.44,427765.85,4427412,3754.13,57144.64,2.19,321.68,...,10012.78,0.023407,0.052002428,0.006947,0.000848,114.00,0.000302,148150.17,0.005282,-0.012627
3,4,ADANIGREEN.NS,187.47,2.00,362547.11,3040135,1644.30,57378.00,1.80,50.08,...,5870.00,0.016191,0.068440866,0.010832,0.000541,1584.00,0.005139,71479.00,-0.031935,-0.041387
4,5,ADANIPORTS.NS,32.84,3.81,333466.14,6743503,8803.01,50060.34,6.06,225.42,...,12488.07,0.037449,0.060172584,0.009033,0.001305,432.03,0.001494,114123.08,0.050741,0.065916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,94,ULTRACEMCO.NS,44.29,3.29,294919.97,324225,6409.56,11459.47,5.77,1944.91,...,11095.25,0.037621,0.258557333,0.010047,0.019769,288.69,0.001017,93991.36,0.036670,0.098200
94,95,MCDOWELL-N.NS,66.05,2.11,83513.12,993566,1271.38,202.80,10.33,93.63,...,1451.70,0.017383,2.48816568,0.006042,0.001280,145.50,0.001733,11024.60,0.069552,0.054309
96,97,VBL.NS,87.65,1.62,185178.24,2058022,2055.92,5431.31,15.68,53.39,...,3688.37,0.019918,-0.147872244,-0.004337,0.000999,649.61,0.003605,15187.18,0.188255,0.003801
97,98,VEDL.NS,21.32,15.72,176933.16,11954138,4980.01,75064.00,7.62,84.96,...,37282.00,0.210712,0.258592668,0.109708,0.000417,372.00,0.003504,193572.00,-0.074551,0.496792


In [5]:
def fetch_momentum_12mo(ticker):
    stock = yf.Ticker(ticker)
    momentum12mo = stock.history(period='12mo')['Close'].pct_change().iloc[-1]
    return {
        'momentum12mo': momentum12mo,
    }    
    
# Fetch financial ratios for each company
momentum_12mo = [fetch_momentum_12mo(ticker) for ticker in result_df['Name']]

# Extract 'momentum3mo' from each dictionary in the financial_ratios list
momentum_12_mo = [momentum_12mo_1['momentum12mo'] for momentum_12mo_1 in momentum_12mo]

# Assign the extracted momentum values to the 'momentum3mo' column in result_df
result_df['momentum12mo'] = momentum_12_mo
result_df

In [7]:
# Define whether higher or lower values are better for each ratio
higher_is_better = {
    'freeCashFlowYield5Yrs': True,
    'bookToMarket': False,
    'momentum12mo': True,
}

# Calculate percentiles for each ratio
for ratio, is_higher_better in higher_is_better.items():
    result_df[f'{ratio} Percentile'] = result_df[ratio].rank(pct=True, ascending=is_higher_better) * 100

In [8]:
# Calculate Composite Score
percentile_columns = [f'{ratio} Percentile' for ratio in higher_is_better.keys()]
result_df['Composite Score'] = result_df[percentile_columns].mean(axis=1)

# Rank companies based on Composite Score
result_df['Rank'] = result_df['Composite Score'].rank(ascending=False).astype(int)

# Display results in ascending order of rank
result_df_sorted = result_df.sort_values(by='Rank')
result_df_sorted.reset_index(drop=True, inplace=True)
result_df_sorted

,S.No.,Name,priceToEarnings,earningsYield,enterpriseValue,volume,profits,debt,roa,bookValue,...,bookToMarket,assets,accrualCF,freeCashFlowYield5Yrs,momentum12mo,freeCashFlowYield5Yrs Percentile,bookToMarket Percentile,momentum12mo Percentile,Composite Score,Rank
0,88,TCS.NS,31.84,4.31,1445683.81,2105634,45580.38,7764.00,29.80,278.27,...,0.000252,154940.00,0.043103,0.116077,0.010900,77.319588,97.938144,71.134021,82.130584,1
1,56,JSWSTEEL.NS,18.54,8.52,272416.58,2405175,10822.44,81874.00,1.84,302.43,...,0.001520,215938.00,0.010412,0.142675,0.021036,84.536082,56.701031,90.721649,77.319588,2
2,55,JINDALSTEL.NS,15.14,7.87,93784.87,2466612,5553.44,13467.35,6.75,406.84,...,0.001195,72212.08,0.064810,0.429901,0.010661,97.938144,62.886598,70.103093,76.975945,3
3,44,HINDALCO.NS,12.51,10.29,161735.66,6359423,9378.45,58372.00,4.53,442.75,...,0.001893,224741.00,-0.000390,0.275511,0.018503,92.783505,47.422680,88.659794,76.288660,4
4,43,HAL.NS,35.24,3.76,195134.45,1600153,6144.30,0.37,8.98,375.12,...,0.001544,71766.40,-0.012752,0.106266,0.027601,75.257732,55.670103,93.814433,74.914089,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,61,M&M.NS,21.67,6.81,331149.95,2699862,11130.81,99711.68,5.51,486.41,...,0.002310,220773.04,0.110392,-0.049604,-0.037094,9.278351,41.237113,1.030928,17.182131,93
93,73,RELIANCE.NS,29.08,5.44,2280004.00,6612950,69878.51,316697.00,4.78,1110.34,...,0.003330,1682215.00,0.051509,-0.048592,-0.016018,10.309278,31.958763,4.123711,15.463918,94
94,34,GAIL.NS,15.83,7.59,142989.16,18613059,8061.56,17951.01,5.49,110.81,...,0.051520,116201.75,0.117123,-0.001432,-0.009976,19.587629,2.061856,9.278351,10.309278,95
95,64,MUTHOOTFIN.NS,14.04,9.19,104616.17,565917,3896.04,57992.34,4.85,578.86,...,0.007337,83093.10,0.070892,-0.183738,-0.011165,5.154639,16.494845,8.247423,9.965636,96


In [11]:
# Select top 50% based on rank
top_50_percent = result_df_sorted.head(int(len(result_df_sorted) * 0.2))
top_50 = top_50_percent['Name']
top_50

0            TCS.NS
1       JSWSTEEL.NS
2     JINDALSTEL.NS
3       HINDALCO.NS
4            HAL.NS
5     BAJAJ-AUTO.NS
6     HEROMOTOCO.NS
7         COLPAL.NS
8      TATASTEEL.NS
9             LT.NS
10    ASIANPAINT.NS
11    TATACONSUM.NS
12         TRENT.NS
13      DIVISLAB.NS
14        MARICO.NS
15     NESTLEIND.NS
16    HINDUNILVR.NS
17           ITC.NS
18           ABB.NS
Name: Name, dtype: object

In [15]:
#Backtesting & Optimization

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from datetime import datetime, timedelta

def fetch_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

def preprocess_data(data):
    data["Return_6months"] = (data["Adj Close"] / data["Adj Close"].shift(126) - 1) * 100
    data["Return_3months"] = (data["Adj Close"] / data["Adj Close"].shift(63) - 1) * 100
    data["Return_1month"] = (data["Adj Close"] / data["Adj Close"].shift(21) - 1) * 100
    data["Return_1week"] = (data["Adj Close"] / data["Adj Close"].shift(5) - 1) * 100
    data["Return_1day"] = (data["Adj Close"] / data["Adj Close"].shift(1) - 1) * 100

    

    return data

nifty50_data = {}
start_date = "2023-01-01"
end_date = "2024-01-01"

# Fetch Nifty50 data
for symbol in top_50:
    nifty50_data[symbol] = fetch_stock_data(symbol, start_date, end_date)

# trade data storage
trade_data = []

# Iterate over the stocks
for symbol, stock_data in nifty50_data.items():
    processed_data = preprocess_data(stock_data)

    # Check if there are enough data points
    if len(processed_data) < 2:
        print(f"Not enough data for {symbol}. Skipping...")
        continue

    # Buy shares on the start date
    initial_capital = 100000
    shares_bought = int(initial_capital / processed_data["Close"].iloc[0])
    trade_data.append({"Symbol": symbol, "Action": "Buy", "Buy Date": processed_data.index[0], "Price": processed_data["Close"].iloc[0], "Shares": shares_bought, "Capital": initial_capital, "Profit/Loss": 0, "Sell Date": None, "Days in Trade": 0})

    # Sell shares on the end date
    capital = shares_bought * processed_data["Close"].iloc[-1]
    profit_loss = capital - initial_capital
    trade_data[-1]["Sell Date"] = processed_data.index[-1]
    trade_data[-1]["Days in Trade"] = len(pd.bdate_range(trade_data[-1]["Buy Date"], trade_data[-1]["Sell Date"]))
    trade_data.append({"Symbol": symbol, "Action": "Sell", "Buy Date": None, "Sell Date": processed_data.index[-1], "Days in Trade": 0, "Price": processed_data["Close"].iloc[-1], "Shares": shares_bought, "Capital": capital, "Profit/Loss": profit_loss})


trade_df = pd.DataFrame(trade_data)
trade_df.to_csv("QVM_.csv", index=False)

# Calculate total initial capital and total final capital
total_initial_capital = trade_df[trade_df["Action"] == "Buy"]["Capital"].sum()
total_final_capital = total_initial_capital + trade_df[trade_df["Action"] == "Sell"]["Profit/Loss"].sum()

# Display the final capital, percentage return, and in-trade days for each stock
for symbol in nifty50_data.keys():
    final_capital = trade_df[(trade_df["Symbol"] == symbol) & (trade_df["Action"] == "Sell")]["Capital"].iloc[0]
    percentage_return = (final_capital - initial_capital) / initial_capital * 100
    in_trade_days = trade_df[(trade_df["Symbol"] == symbol) & (trade_df["Action"] == "Sell")]["Days in Trade"].iloc[0]
    print(f"Symbol: {symbol}, Final Capital: {final_capital:.2f}, Percentage Return: {percentage_return:.2f}%, In-Trade Days: {in_trade_days}")

# Define the symbol for Nifty50
nifty50_symbol = "^NSEI"

# Download Nifty50 historical data
nifty50_data = yf.download(nifty50_symbol, start=start_date, end=end_date)

# Calculate total return
start_value = nifty50_data["Adj Close"].iloc[0]
end_value = nifty50_data["Adj Close"].iloc[-1]
total_return = ((end_value / start_value) - 1) * 100

# Display the total percentage return across all stocks
total_percentage_return = (total_final_capital - total_initial_capital) / total_initial_capital * 100
print(f"Total Initial Capital: {total_initial_capital:.2f}")
print(f"Total Final Capital Across All Stocks: {total_final_capital:.2f}")
print(f"Total Percentage Return Across All Stocks: {total_percentage_return:.2f}%")
# Display the total return of Nifty50
print(f"Total Return of Nifty50: {total_return:.2f}%")

# trade data storage
y_trade_data = []
nifty_stocks = ["^NSEI"]    
# Iterate over the years
years = range(int(start_date[:4]), int(end_date[:4]))
for year in years:
    # Fetch stock data for each stock
    stock_data_dict = {}
    for symbol in top_50:
        stock_data_dict[symbol] = fetch_stock_data(symbol, f"{year}-01-01", f"{year}-12-31")

    #yearly returns
    yearly_returns = {}

    # Iterate over the stocks
    y_total_portfolio_value = 0
    y_total_initial_capital = 0
    for symbol, stock_data in stock_data_dict.items():
        # Check if there are enough data points
        if len(stock_data) < 2:
            print(f"Not enough data for {symbol} in {year}. Skipping...")
            continue

        # Buy shares on the start date
        y_initial_capital = 100000
        y_shares_bought = int(y_initial_capital / stock_data["Close"].iloc[0])
        y_capital = y_shares_bought * stock_data["Close"].iloc[-1]
        y_total_portfolio_value += y_capital
        y_total_initial_capital += y_initial_capital
   
    # Fetch stock data for each stock
    n_stock_data_dict = {}

    for symbol in nifty_stocks:
        n_stock_data_dict[symbol] = fetch_stock_data(symbol, f"{year}-01-01", f"{year}-12-31")

    # nifty yearly returns
    n_yearly_returns = {}

    # Iterate over the stocks
    n_total_portfolio_value = 0
    n_total_initial_capital = 0
    for symbol, stock_data in n_stock_data_dict.items():
        # Check if there are enough data points
        if len(stock_data) < 2:
            print(f"Not enough data for {symbol} in {year}. Skipping...")
            continue

        # Buy shares on the start date
        n_initial_capital = 100000
        n_shares_bought = int(n_initial_capital / stock_data["Close"].iloc[0])
        n_capital = n_shares_bought * stock_data["Close"].iloc[-1]
        n_total_portfolio_value += n_capital
        n_total_initial_capital += n_initial_capital

    # Append year and yearly return for the portfolio
    yearly_returns["Year"] = year
    yearly_returns["Nifty_50"] = ((n_total_portfolio_value - n_total_initial_capital) / n_total_initial_capital) * 100
    yearly_returns["Portfolio"] = ((y_total_portfolio_value - y_total_initial_capital) / y_total_initial_capital) * 100

    # Append yearly returns to trade data
    y_trade_data.append(yearly_returns)


y_trade_df = pd.DataFrame(y_trade_data)


# Calculate CAGR (Compound Annual Growth Rate)
start_year = int(start_date[:4])
end_year = int(end_date[:4])
years = end_year - start_year

# CAGR formula: ((final value / beginning value) ^ (1 / no of years)) - 1
cagr = ((total_final_capital / total_initial_capital) ** (1 / years)) - 1
print(f"CAGR (Compound Annual Growth Rate of Portfolio): {cagr:.2%}")

# Calculate and print Nifty CAGR
nifty_cagr = ((end_value / start_value) ** (1 / years)) - 1
print(f"CAGR (Compound Annual Growth Rate of Nifty): {nifty_cagr:.2%}")

#Portfolio optimization
#Define the stock tickers and historical data
stock_tickers = top_50
start_year = int(start_date[:4])
end_year = int(end_date[:4])

# Function to fetch historical stock data
def fetch_historical_data(stock_ticker, start_date, end_date):
    return yf.download(stock_ticker, start=start_date, end=end_date)['Adj Close']

#Function to calculate portfolio returns for a given year
def calculate_portfolio_metrics(weights, year):
    #Fetch historical data for each stock
    stock_data = pd.DataFrame({ticker: fetch_historical_data(ticker, f"{year}-01-01", f"{year+1}-01-01") for ticker in stock_tickers})

    #Calculate portfolio returns
    portfolio_returns = (stock_data.pct_change() * weights).sum(axis=1).dropna()

    #Calculate portfolio cumulative returns
    cumulative_returns = (portfolio_returns + 1).cumprod()

    #Calculate total return for the year
    total_return = cumulative_returns.iloc[-1] - 1

    return total_return, weights  #Return the total return and weights for the year

#Initialize empty dictionary to store annual returns and weights
annual_returns_and_weights = {}

#Loop through each year
for year in range(start_year, end_year):
    #Fetch historical stock data
    stock_data = pd.DataFrame({ticker: fetch_historical_data(ticker, f"{year}-01-01", f"{year+1}-01-01") for ticker in stock_tickers})

    #Calculate expected returns and sample covariance matrix of asset returns
    mu = expected_returns.mean_historical_return(stock_data)
    cov_matrix = risk_models.sample_cov(stock_data)

    #Optimize for maximum Sharpe ratio
    ef = EfficientFrontier(mu, cov_matrix)
    weights = ef.max_sharpe(risk_free_rate= 0.06)  #Assuming a risk-free rate of -3%

    #Calculate portfolio metrics for the year using the optimal weights
    annual_returns_and_weights[year] = calculate_portfolio_metrics(weights, year)

#Print annual returns and weights
for year, (return_value, weights) in annual_returns_and_weights.items():
    print(f"Year: {year}")
    print(f"Return: {return_value:.2%}")
    print("Weights:")
    for ticker, weight in weights.items():
        print(f"{ticker}: {weight:.2%}")
    print()

#Display trade data
print(trade_df)
print(y_trade_df)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
#Passive portfolio

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import pandas as pd
import yfinance as yf
import numpy as np

# Define the stock tickers and historical data
stock_tickers = top_50
start_year = int(start_date[:4])
end_year = int(end_date[:4])

# Function to fetch historical stock data
def fetch_historical_data(stock_ticker, start_date, end_date):
    return yf.download(stock_ticker, start=start_date, end=end_date)['Adj Close']

# Fetch historical stock data for the first year
first_year_stock_data = pd.DataFrame({ticker: fetch_historical_data(ticker, f"{start_year}-01-01", f"{start_year+1}-01-01") for ticker in stock_tickers})

# Calculate expected returns and sample covariance matrix of asset returns for the first year
mu = expected_returns.mean_historical_return(first_year_stock_data)
cov_matrix = risk_models.sample_cov(first_year_stock_data)

# Optimize for maximum Sharpe ratio for the first year
ef = EfficientFrontier(mu, cov_matrix)
weights = ef.max_sharpe(risk_free_rate=-0.5)  # Assuming a risk-free rate of -0.5% for demonstration

# Initialize empty dictionary to store annual returns
annual_returns = {}

# Loop through each year
for year in range(start_year, end_year):
    # Fetch historical stock data for the year
    stock_data = pd.DataFrame({ticker: fetch_historical_data(ticker, f"{year}-01-01", f"{year+1}-01-01") for ticker in stock_tickers})

    # Calculate portfolio returns for the year using the optimized weights
    portfolio_returns = (stock_data.pct_change() * weights).sum(axis=1).dropna()

    # Calculate portfolio cumulative returns
    cumulative_returns = (portfolio_returns + 1).cumprod()

    # Calculate total return for the year
    total_return = cumulative_returns.iloc[-1] - 1

    # Store the total return for the year
    annual_returns[year] = total_return

# Print weights of the stocks
print("Weights of the stocks in the portfolio:")
for ticker, weight in weights.items():
    print(f"{ticker}: {weight:.2%}")

# Print annual returns with the same weights for each year
print("\nAnnual Returns:")
for year, return_value in annual_returns.items():
    print(f"Year: {year}, Return: {return_value:.2%}")


In [ ]:
#Weightage for next year  and returns for the year

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import pandas as pd
import yfinance as yf

# Define the stock tickers and historical data range
stock_tickers = top_50
#start_date = "2010-01-01"
#end_date = "2023-12-31"

# Function to fetch historical stock data
def fetch_historical_data(stock_ticker, start_date, end_date):
    return yf.download(stock_ticker, start=start_date, end=end_date)['Adj Close']

# Initialize empty dictionary to store historical stock data
stock_data = {}

# Fetch historical stock data for each stock
for ticker in stock_tickers:
    stock_data[ticker] = fetch_historical_data(ticker, start_date, end_date)

# Combine stock data into a single DataFrame
all_stock_data = pd.DataFrame(stock_data)

# Calculate expected returns and sample covariance matrix of asset returns
mu = expected_returns.mean_historical_return(all_stock_data)
cov_matrix = risk_models.sample_cov(all_stock_data)

# Optimize for maximum Sharpe ratio
ef = EfficientFrontier(mu, cov_matrix)
weights = ef.max_sharpe()

# Print optimal weights for next year 
print("Optimal Weights for next year:")
for ticker, weight in weights.items():
    print(f"{ticker}: {weight:.2%}")

# Determine start date and end date for next year
next_year_start_date = pd.Timestamp(end_date) + pd.DateOffset(days=1)
next_year_end_date = pd.Timestamp(end_date) + pd.DateOffset(years=1)

# Fetch historical stock data for next year if available
next_year_stock_data = {}
for ticker in stock_tickers:
    next_year_stock_data[ticker] = fetch_historical_data(ticker, next_year_start_date, next_year_end_date)

# Check if data for next year exists
if all(data.empty for data in next_year_stock_data.values()):
    print("No historical data available for next year.")
else:
    # Combine next year stock data into a single DataFrame
    all_next_year_stock_data = pd.DataFrame(next_year_stock_data)
    
    # Calculate portfolio returns for next year using optimal weights
    portfolio_returns = (all_next_year_stock_data.pct_change() * weights).sum(axis=1).dropna()
    # Calculate total portfolio returns for next year
    total_portfolio_return_next_year = portfolio_returns.sum()

    # Print total portfolio returns for next year
    print("Total Portfolio Returns for next year:", total_portfolio_return_next_year)


In [ ]:
#weightage of stocks for next month and returns for the month

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import pandas as pd
import yfinance as yf

# Define the stock tickers and historical data
stock_tickers = top_50
#start_date = "2015-01-01"
#end_date = "2024-01-31" 

# Function to fetch historical stock data
def fetch_historical_data(stock_ticker, start_date, end_date):
    return yf.download(stock_ticker, start=start_date, end=end_date)['Adj Close']

# Fetch historical stock data up to the end month
stock_data = pd.DataFrame({ticker: fetch_historical_data(ticker, start_date, end_date) for ticker in stock_tickers})

# Calculate expected returns and sample covariance matrix of asset returns
mu = expected_returns.mean_historical_return(stock_data)
cov_matrix = risk_models.sample_cov(stock_data)

# Optimize for maximum Sharpe ratio
ef = EfficientFrontier(mu, cov_matrix)
weights = ef.max_sharpe(risk_free_rate=-0.6)

# Print the optimal weights for the following month after the end date
print("Optimal Weights for the Following Month after the End Date:", pd.Timestamp(end_date).to_period('M').asfreq('M').to_timestamp().strftime("%Y-%m"))
print("Weights:")
for ticker, weight in weights.items():
    print(f"{ticker}: {weight:.2%}")


# Determine the start and end dates for the next month
next_month_start_date = pd.Timestamp(end_date) + pd.offsets.MonthBegin(1)
next_month_end_date = pd.Timestamp(next_month_start_date) + pd.offsets.MonthEnd(0)

# Fetch historical stock data for the next month if available
try:
    next_month_stock_data = pd.DataFrame({ticker: fetch_historical_data(ticker, next_month_start_date, next_month_end_date) for ticker in stock_tickers})

    # Calculate portfolio returns for the next month using optimal weights
    portfolio_returns_next_month = (next_month_stock_data.pct_change() * weights).sum(axis=1).dropna()

    # Calculate total portfolio returns for the next month
    total_portfolio_return_next_month = portfolio_returns_next_month.sum()

    # Print total portfolio returns for the next month
    print("Total Portfolio Returns for the Next Month:", total_portfolio_return_next_month)

except Exception as e:
    # Display optimal weights if stock data is not available for the next month
    print("Stock data for the next month is not available. Displaying optimal weights instead:")
    print("Optimal Weights for the Following Month after the End Date:", pd.Timestamp(end_date).to_period('M').asfreq('M').to_timestamp().strftime("%Y-%m"))
    print("Weights:")
    for ticker, weight in weights.items():
        print(f"{ticker}: {weight:.2%}")
